In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import shutil
import tensorflow as tf
from tensorflow.keras import Input

In [ ]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.1


In [ ]:
data_dir = "/content/dataset"
lung_cancer_dir = os.path.join(data_dir, "lung_cancer")
normal_dir = os.path.join(data_dir, "normal")

In [ ]:
os.makedirs(lung_cancer_dir, exist_ok=True)
os.makedirs(normal_dir, exist_ok=True)

In [ ]:
for filename in os.listdir(data_dir):
    if filename.endswith(".png"):
        if "000019" in filename or "000020" in filename:
            shutil.move(os.path.join(data_dir, filename), lung_cancer_dir)
        elif "000021" in filename:
            shutil.move(os.path.join(data_dir, filename), normal_dir)

In [ ]:
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = data_gen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary',
    subset='training'
)

val_data = data_gen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)

Found 158 images belonging to 4 classes.
Found 39 images belonging to 4 classes.


In [ ]:
print(os.listdir('/content/dataset'))
print(os.listdir('/content/dataset/lung'))

['normal', 'lung_cancer', '.ipynb_checkpoints', 'lung']
['000230_05_01_165.png', '000103_02_01_060.png', '000103_01_01_173.png', '000223_08_01_109.png', '000103_12_02_161.png', '000238_09_03_162.png', '000138_02_01_101.png', '000021_03_01_017.png', '000103_06_01_131.png', '002463_01_01_029.png', '000138_01_01_148.png', '000103_08_01_016.png', '000019_03_01_025.png', '000284_01_02_105.png', '000103_09_01_019.png', '000020_04_01_143.png', '000315_04_03_145.png', '000027_02_01_105.png', '000198_05_01_124.png', '000103_12_01_105.png', '000328_08_03_180.png', '000021_02_01_068.png', '000017_02_01_016.png', '000143_04_02_126.png', '000230_05_01_233.png', '000223_09_02_132.png', '000874_03_02_219.png', '000278_07_02_182.png', '000747_03_03_172.png', '000276_05_02_188.png', '000103_01_01_104.png', '000247_05_01_178.png', '000308_03_02_091.png', '000328_08_03_167.png', '003412_03_01_108.png', '002639_05_01_141.png', '000223_07_01_106.png', '000223_08_01_106.png', '000207_04_01_191.png', '004085

In [ ]:
model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 900ms/step - accuracy: 0.9482 - loss: 0.1734 - val_accuracy: 1.0000 - val_loss: 3.3967e-29
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 671ms/step - accuracy: 1.0000 - loss: 1.3371e-28 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 653ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 858ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 659ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 846ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 896ms/step - accuracy

In [ ]:
model.save("lung_cancer_detector.h5")

In [ ]:
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    if prediction[0] > 0.5:
        print("Prediction: Lung Cancer Detected")
    else:
        print("Prediction: No Lung Cancer Detected")

In [ ]:
predict_image("/content/dataset/lung/000001_03_01_088.png")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Prediction: Lung Cancer Detected
